In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display, clear_output
import matplotlib.widgets
import ipywidgets as widgets
from matplotlib import animation, rc
import matplotlib.patches as patches
%matplotlib inline

# functions that will used to interfere two waves at the second beam splitter
# interfere_amplitudes calcualtes the effective final amplitude given two waves with their amplitudes and phases
# interfere_phases calcualtes the effective final phase given two waves with their amplitudes and phases
def interfere_amplitudes(a1,phi1,a2,phi2):
    return np.sqrt(np.power((a1*np.sin(phi1)+a2*np.sin(phi2)),2)+np.power((a1*np.cos(phi1)+a2*np.cos(phi2)),2))

def interfere_phases(a1,phi1,a2,phi2):
    return np.arctan((a1*np.sin(phi1)+a2*np.sin(phi2))/(a1*np.cos(phi1)+a2*np.cos(phi2)))

fig = plt.figure()
ax = plt.axes()
lx1,=ax.plot([], [], 'r')
lx2,=ax.plot([], [], 'r')
lx3,=ax.plot([], [], 'b')
lx4,=ax.plot([], [], 'r')
lx5,=ax.plot([], [], 'r')
lx6,=ax.plot([], [], 'r')
ly1,=ax.plot([], [], 'r')
ly2,=ax.plot([], [], 'r')
ly3,=ax.plot([], [], 'r')  
patch = patches.Rectangle((0, 0), 0, 0, facecolor='white',edgecolor='black')
glass=ax.add_patch(patch)
plt.close()

def create_figure(c,t,width,xglass,eta,k,Amp,t1,t2):
    r1 = np.sqrt(1-t1*t1); # reflectivity of the first beam splitter
    r2 = np.sqrt(1-t2*t2); # reflectivity of the first beam splitter
    x1 = np.linspace(0, 1,100)
    x2 = np.linspace(1, xglass,100)
    x3 = np.linspace(xglass, xglass+width,100)
    x4 = np.linspace(xglass+width,5,100)
    x5 = np.linspace(1,5,100)
    x6 = np.linspace(5,6,100)
    y1 = np.linspace(0,4,100)
    y2 = np.linspace(0,4,100)
    y3 = np.linspace(4,5,100)
    
    # creating the glass slide
    glass.set_width(width)
    glass.set_height(1.25)
    glass.set_xy([xglass,-0.625])
    
    # creating incoming sine wave    
    lx1.set_data(x1, Amp*np.sin(k*(x1-c*t)))

    # creating the wave transmitted through the first beam splitter    
    lx2.set_data(x2, t1*Amp*np.sin(k*(x2-c*t)))

    # creating the wave travelling inside the glass slide    
    lx3.set_data(x3,t1*Amp*np.sin(eta*k*(x3-xglass-c*t/eta)+k*xglass))

    # creating the wave emerging from the glass slide and htiing mirror m2    
    lx4.set_data(x4,t1*Amp*np.sin(k*(x4-c*t) + (eta-1)*k*width))

    # creating the wave reflecting off the first beamsplitter (pi phase shift will occur on reflection here)    
    ly1.set_data(1 + r1*Amp*np.sin(k*(y1-0-c*t) + np.pi + k*1),y1)

    # creating the wave reflecting off the mirror m2 (pi phase shift will occur on reflection here)    
    ly2.set_data(5 + t1*Amp*np.sin(k*(y1-0-c*t) + np.pi + k*5 + (eta-1)*k*width ),y1)

    # creating the wave reflecting off the mirror m1 (pi phase shift will occur on reflection here)    
    lx5.set_data(x5,4+r1*Amp*np.sin(k*(x5-c*t) + k*(4-0) + 2*np.pi))

    # creating the wave emerging on the horizontal path after the second beam splitter 
    #(pi phase shift occurs for the wave in the lower path that is reflected to the right)    
    lx6.set_data(x6,4 + Amp*interfere_amplitudes(r1*t2,k*9,t1*r2,k*9 +(eta-1)*k*width)*np.sin(k*(x6-5-c*t)+interfere_phases(r1*t2,k*9,t1*r2,k*9 +(eta-1)*k*width)))

    # creating the wave emerging on the vertical path after the second beam splitter 
    #(NO phase shift occurs when the wave moving right is reflected upwards by the beam splitter. 
    # This is because we had chosen the convention that the second beam splitter has mirrored surface on the right hand side)
    ly3.set_data(5+Amp*interfere_amplitudes(r1*r2,k*9,t1*t2,k*9 +(eta-1)*k*width+np.pi)*np.sin(k*(y3-4-c*t)+interfere_phases(r1*r2,k*9,t1*t2,k*9 +(eta-1)*k*width+np.pi)),y3)
    
    return glass,lx1,lx2,lx3,lx4,lx5,lx6,ly1,ly2,ly3

def background():
        # the interferometer paths Syntax: (xdata), (ydata) for the various points
        up = plt.Line2D((1,6),(4,4),color="whitesmoke")
        do = plt.Line2D((0,5),(0,0),color="whitesmoke")
        le = plt.Line2D((1,1),(0,4),color="whitesmoke")
        ri = plt.Line2D((5,5),(0,5),color="whitesmoke")


        #mirrors
        m1 = plt.Line2D((0.5,1.5),(3.5,4.5),color="black") # mirror at top left corner
        m2 = plt.Line2D((4.5,5.5),(-0.5,0.5),color="black") # mirror at bootom right corner

        # beam spitters  (For calcualtions, first beam splitter will have reflecting surface on the left side, Seond will have reflectivng surface on the right side)
        bs1 = plt.Line2D((0.5,1.5),(-0.5,0.5),color="grey",linestyle='dashed',lw=2.5)
        bs2 = plt.Line2D((4.5,5.5),(3.5,4.5),color="grey",linestyle='dashed',lw=2.5)


        # adding elements to the figures
        ax.add_line(up)
        ax.add_line(do)
        ax.add_line(le)
        ax.add_line(ri)
        ax.add_line(m1)
        ax.add_line(m2)
        ax.add_line(bs1)
        ax.add_line(bs2)
        
        # Axis scaling turned on so that aspect ratio is maintained
        ax.axis("scaled")
        # Axis turned off 
        ax.axis("off")
        
#def glass(width,xglass):      
        # the glass slide in the lower arm of the Mach-Zehnder
 #       rectangle = plt.Rectangle((xglass,-0.125*3),width=width,height=2*0.125*3,facecolor='white',edgecolor='black')
  #      ax.add_patch(rectangle)
    
# testing background and glass    
background() 
#glass(0.57,2.5)

def f(c,width,xglass,eta,k,Amp,t1,t2):
    return c,width,xglass,eta,k,Amp,t1,t2
w = widgets.interactive(f,c = widgets.FloatSlider(value=0.2,min=0,max=0.5,step=0.1,description='Speed:',continuous_update=False),
                            width = widgets.FloatSlider(value=0.57,min=0.01,max=1,step=0.01,description='Width:',continuous_update=False),
                       xglass = widgets.FloatSlider(value=2.5,min=1.5,max=3.5,step=0.1,description='xglass :',continuous_update=False),
                       eta =widgets.FloatSlider(value=3,min=1,max=3,step=0.01,description='eta:',continuous_update=False),
                       k= widgets.FloatSlider(value=5,min=1,max=30,step=0.1,description='k:',continuous_update=False),
                       Amp= widgets.FloatSlider(value=0.13,min=0.062,max=0.25,step=0.001,description='A:',continuous_update=False),
                       t1= widgets.FloatSlider(value=0.707,min=0.001,max=0.999,step=0.001,description='t1:',continuous_update=False),
                       t2 = widgets.FloatSlider(value=0.707,min=0.001,max=0.999,step=0.001,description='t2',continuous_update=False));
display(w)

btn = widgets.Button(description='Set paramters')
clear_btn = widgets.Button(description='Clear output')
global out
out = widgets.Output()


def btn_eventhandler(obj):
    c,width,xglass,eta,k,Amp,t1,t2 = w.result
    def init():        
        lx1.set_data([], [])
        lx2.set_data([], [])
        lx3.set_data([], [])
        lx4.set_data([], [])
        lx5.set_data([], [])
        lx6.set_data([], [])
        ly1.set_data([], [])
        ly2.set_data([], [])
        ly3.set_data([], [])
        glass.set_width(0)
        glass.set_height(0)
        glass.set_xy([0,0])
        
        return glass,lx1,lx2,lx3,lx4,lx5,lx6,ly1,ly2,ly3     
        
    def animate(i): 
        return create_figure (c,i,width,xglass,eta,k,Amp,t1,t2)
    global anim
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                  frames=100, interval=100, blit=True,repeat=True)
    
    out.append_display_data(HTML(anim.to_jshtml()))#anim.to_html5_video()))

def clear_btn_eventhandler(obj):
    out.clear_output()
        
display(btn)
display(clear_btn)
btn.on_displayed(btn_eventhandler)  

btn.on_click(btn_eventhandler)
clear_btn.on_click(clear_btn_eventhandler)

display(out)

interactive(children=(FloatSlider(value=0.2, continuous_update=False, description='Speed:', max=0.5), FloatSli…

Button(description='Set paramters', style=ButtonStyle())

Button(description='Clear output', style=ButtonStyle())

Output()